# 9. Flask - Final Preparation and Mission Deployment

Our Mission Control is fully functional in the simulator (`localhost`), but now it's time for the final step: launching our application into the reality of the internet. In this final lesson, we will prepare our code for the production environment, improve its appearance, and perform the actual deployment, so that our application is accessible to anyone, from anywhere.

- Improving the appearance with `Bootstrap`
- Modifying the `configuration` for deployment
- Application `deployment` & `hosting` options

## 9.1. Rebuilding for a Production Machine
The development environment is designed for convenience, while production is for **security, performance, and stability**. We must therefore make several key adjustments.

### 9.1.1. Engine Swap: Production WSGI Server
The development server (`app.run()`) is not suitable for production. Instead, we will deploy **Gunicorn**, a robust and proven WSGI server.

1.  **Installing Gunicorn:**
    ```bash
    pip install gunicorn
    ```
2.  **Component List (`requirements.txt`):** The hosting service needs to know which libraries to install for our application. We will create a list:
    ```bash
    pip freeze > requirements.txt
    ```
3.  **Startup Sequence (`Procfile`):** We need to tell the hosting service how to start our application. We will create a file named `Procfile` (without an extension) in the root directory of the project with the following content:
    ```
    web: gunicorn wsgi:app
    ```
    *This command says: "For the web process (`web`), start the `gunicorn` server and tell it to find the `app` instance in the `wsgi.py` file."*

### 9.1.2. Securing Configuration: Environment Variables and `.env`
In production, we never leave sensitive data - like passwords and access credentials (e.g., `SECRET_KEY`) - written directly in the code. Therefore, we will use **environment variables** and an `.env` file to manage them. We will install `python-dotenv`.

1.  **Installing `python-dotenv`:**
    ```bash
    pip install python-dotenv
    ```
    
2.  **`.env` file:** In the root directory of the project, create a file named `.env`. **Never add this file to Git!** Add `.env` to your `.gitignore`.

    ```
    # FILE: .env
    # These are variables for the production environment

    SECRET_KEY=my_secret_password
    DATABASE_URL=sqlite:///production_database.db # Or another DB
    
    ```

3.  **Upgrading `config.py`:** we will modify our `Config` class:

    ```python
    # FILE: configuration.py
    import os # import OS
    from dotenv import load_dotenv # import load_dotenv

    load_dotenv()

    class Config:
        """Base configuration."""
        STATIC_FOLDER = 'static'
        TEMPLATE_FOLDER = 'templates'
        SECRET_KEY = os.environ.get('SECRET_KEY')

        SQLALCHEMY_TRACK_MODIFICATIONS = False
        SQLALCHEMY_DATABASE_URI = os.environ.get("DATABASE_URL")

    class ProdConfig(Config):
        APP_ENV = 'production'
        DEBUG = False

    class DevConfig(Config):
        APP_ENV = 'development'
        DEBUG = True
    ```

## 9.2. Improving Appearance: Bootstrap Integration
To make our information panel look professional, we will use the CSS framework **Bootstrap** and can leverage the capabilities of an **extension for Flask**.

1. **Installing the extension:**
Documentation: https://bootstrap-flask.readthedocs.io/en/stable/basic/

```bash
    pip install bootstrap-flask
```

2. **Initialization in the application factory (app/`__init__.py`):**

In [ ]:
# __init__.py

import os
from flask import Flask
from dotenv import load_dotenv
from .config.configuration import ProdConfig
from flask_bootstrap import Bootstrap5 # import Bootstrap


load_dotenv() # loads values from .env

def create_app():

    app = Flask(__name__)

    # CONFIGURATION - using ENV

    # PLUGIN INIT - extensions for "app" object
    bs = Bootstrap5(app) # connects "Flask app" and "Bootstrap"
    
    # REGISTRATION (Blueprints / Routes)

    # APP CONTEXT - where we need to know the structure of the app 

    return app

3. **Linking with the base template (base.html):**
- Thanks to modifying the "base" template, from which other HTML files inherit, the changes will be immediately reflected in all descendants.

In [ ]:
<!-- base.html -->

<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>{% block page_title %} Title {% endblock page_title %}</title> <!-- místo pro dynamický obsah -->
    
    {{ bootstrap.load_css() }} <!-- Bootstrap CSS -->
</head>

<body>
    {% include 'header.html' %} <!-- zde vložíme obsah z header.html -->
    {% block page_content %} Content {% endblock page_content %} <!-- místo pro dynamický obsah -->
    {% include 'footer.html' %} <!-- zde vložíme obsah z footer.html -->

    {{ bootstrap.load_js() }} <!-- Bootstrap JS -->
</body>

</html>

Now we can use all **CSS classes** and **HTML elements** and components in our templates (for example, navigation, main page layout (heroes), footer and header sections, as well as elements like buttons ..)

- see documentation: https://getbootstrap.com/docs/5.3/getting-started/introduction/

### Bootstrap principle:
We can copy the **Bootstrap HTML code** of selected elements:
- from https://getbootstrap.com/docs/5.3/examples/

And paste it into the appropriate templates of our project (again, we will use inheritance = we will primarily edit the code in the base, header, and footer templates from which our other pages "inherit"). Further HTML modifications can, of course, be made on a specific page.
This way, we can quickly create a **professional look**.

## 9.3. Pre-launch Check: Git & GitHub
For deployment (hosting), we will need two things: 
1. **GitHub account**, where we will store the final plans for our application
2. **hosting service**, which will take these plans and build a functional, publicly accessible server from them

* **GitHub:** Acts as our central version control repository for source code. Most modern hosting platforms can integrate directly with your GitHub repository to automatically deploy the application whenever a change is made. This process is called `CI/CD` (Continuous Integration/Continuous Deployment). It represents software automation where code is written, tested, approved, pushed to a remote server (like GitHub), and instantly deployed to the live application.

## 9.4. Mission Start: Deploying the Application

The time has come to launch our application from the local simulator into the real orbit – the internet. For deployment, there are many "spaceports" (hosting platforms), each with its own advantages.

* **PythonAnywhere:** 
    - Ideal for beginners
    - We just need to bring `any .py file` with the "app" object and `requirements.txt`
    - ! It has its own built-in server (uWSGI) that runs everything !
    - The free version offers hosting for 1 project
    - Focused purely on Python (applications, scripts...)
    - No GitHub required
    - Allows managing and editing the application directly in the web browser

* **Render.com:** 
    - Ideal for developing both personal and professional projects
    - We just need to bring `wsgi.py` with the "app" object and `requirements.txt`
    - ! We must "bring" the server (e.g., gunicorn) ourselves (defined in `requirements.txt`) !
    - The free version offers hosting for 1 project
    - Supports most programming languages, databases, and frameworks
    - Integrates with GitHub
    
- další možnosti: 
    - Heroku, DigitalOcean App Platform, Fly.io, Vercel, Railway ..
    - We need `wsgi.py` with the "app" object, `requirements.txt` and a "starter" configuration for our chosen server: a `Procfile` 

Pro naši misi si ukážeme postup na platformě **PythonAnywhere:** a **Render.com**.

### 9.4.1. Nasazení Step by step (PythonAnywhere.com)

* Prepare the final version of our project
* Create an account at: https://www.pythonanywhere.com/
* In the **top navigation bar** select "Web" and choose "Add a new Web App".
* Select the framework (e.g., Flask) and Python version (e.g., the latest available).

**1. Pre-launch Parameters:**
```
Quickstart new Flask project
Enter a path for a Python file you wish to use to hold your Flask app. 
If this file already exists, its contents will be overwritten with the new app.
```
* Default settings for a new project:
    - /home/User_Name/mysite/flask_app.py
        - "mysite" - project name
        - "flask_app.py" - the file where our "app" is available

* We can rename both the project and the file containing the "app" object according to our needs:
    - e.g. : /home/User_Name/Project_Deploy_My_App/app.py
        1. a `project directory` named "Project_Deploy_My_App" will be created. Inside, a pre-prepared app.py file will be generated, where the variable (object) "app" is expected to exist.
        2. a `configuration file "wsgi"` will be created, pre-configured to import the "app" object from "app.py" to run our web app

**Files Section:**
* Navigate to our project folder ("Project_Deploy_My_App").
    - Replace the **app.py** file on PythonAnywhere with **our own file** containing the "app" object, **or paste our code into it** (ensuring it contains the "app" object, for example, by calling the "application factory" function).
    - Upload all other files and folders of our Flask application.

**Web Section:**
- In the top navigation bar, select "Web" and click "Reload" to apply all changes.

- If we need to work with additional extensions and specific versions, we must install them into the built-in venv (virtual environment).
- Follow the instructions here: https://help.pythonanywhere.com/pages/VirtualEnvForWebsites

**Launch!**
- In the "Web" section, launch our deployed application (the link is located above the Reload button): **User_Name.pythonanywhere.com**

### 9.4.2. Deployment Step-by-Step (Render.com):

* Push the final version of your project (especially `wsgi.py` containing the app object, `requirements.txt` – including the selected server – and, of course, other application files) to a GitHub repository.
* Sign up and connect your GitHub account.
* In the main **Dashboard** click on **"New"** and select **"Deploy a Web Service"**.
* Select the GitHub repository containing the application.

**Pre-launch Parameters:**
Render will attempt to configure most things automatically. However, to be safe, check or complete the following details in the settings (**side navigation bar**):
* **Settings Section:**
    * **Name:** unique project name
    
* **Build & Deploy Section:**
    - Repository: the repository with our application
    - Branch: main - main – the branch from our repository (leave default)
    - Git Credentials - our details (leave default)
    - Root Directory - (leave default)
    - Build Filters - (leave default)
    1. **Build Command:** `pip install -r requirements.txt` – enter the command to install libraries and dependencies
    - Pre-Deploy Command - (leave default)
    2.  **Start Command:** `gunicorn wsgi:app` – enter the command for the gunicorn server = start the `gunicorn` server and tell it to find the `app` in `wsgi.py` file
* **Environment Section:**
    3. **Environment Variables:** - insert your values for `SECRET_KEY`, etc. - **Never put sensitive data directly into the code on GitHub!**

**Launch!**
* Click the **"Create Web Service"** button and watch the launch sequence progress in the "Logs" tab – within a few minutes, our project will be online!

# 10. Flask Course Summary
We have successfully completed basic training! We have mastered all key operations and are now ready for independent missions.

## 10.1. The Philosophy of Flask:
Flask provides a functional minimum; we add components only when our project truly requires them and exactly when they are needed.

- We have full control over the project because we are not overwhelmed by a multitude of options and configurations for every possible feature.

- Since there is no enforced application structure, we are responsible for defining it ourselves.

## 10.2 Mission Outcomes: Our New Skills
By completing this course, we have acquired a comprehensive skill set for developing modern web applications.

* We are now able to:
- Create `dynamic web pages` using **Flask** and the **Jinja2** templating system.

- `Handle user input` using robust **WTF forms**.

- `Work with databases` (create, read, update, and delete data) using **SQLAlchemy**.

- Structure code into a `scalable architecture` using **Application Factory** and **Blueprints**.

- Design and build simple `REST APIs` for **machine-to-machine communication**.

- `Prepare and deploy` the application to a **production environment**.

Thanks to this, we are capable of building `dynamic web applications/sites` (such as blogs or portfolios), `data management applications` (user or product registries), or a `backend system with a REST API` for mobile or JavaScript applications.


## 10.3. Next Frontiers (Tips for Improvement)
1. **Increase System Robustness:** 
    - Focus on application quality and reliability by exploring automated testing (e.g., using **pytest**) to verify system functionality after every change. Also, consider implementing further security measures (authentication and authorization).

2. **Publish Your Missions:** 
    - Your projects shouldn't remain classified. Create a portfolio on GitHub, upload your code, and try deploying a selected application (Render, etc.).

3. **Become a Chronicler of Your Expeditions:** 
    - Document the development process, the challenges you faced, and your solutions. Create and maintain README files on GitHub to serve as your **captain's log**.

4. **Update Your Technology:** 
    - Technology is constantly evolving. Occasionally, return to your old code with your new knowledge and perform **refactoring**.

### **Project (Homework): Launch Sequence**

**Mission:** Our control system has successfully passed all simulations in the local environment. Before the final launch into orbit (deployment to a production server), we must perform the last critical adjustments: secure the configuration, improve the appearance, and prepare the startup sequence for the production engines. The goal is to transform our development prototype into a robust, secure, and professional-looking application ready for real-world operation.

`základní struktura naší aplikace`:
- my_project/
    - wsgi.py
    - app/
        - **Procfile**
        - **requirements.txt**
        - **.env**
        - _ _ init _ _.py
        - database.py
        - models/
            - _ _ init _ _.py
            - crewmember.py
        - data/
            - _ _ init _ _.py
            - asteroid.py
            - crew.py
        - templates/
        - static/
        - config/
        - main/ 
            - _ _ init _ _.py
            - routes.py 
            - templates/ 
        - crew/
            - _ _ init _ _.py
            - routes.py 
            - forms.py
            - templates/
        - api/ 
            - _ _ init _ _.py
            - routes.py
        - /instance
            - my_database.db


- The file with our **SQLite DB** is in the default **instance** folder next to our application, which is the **root** folder.


1.  **Secure Configuration:** Modify `config.py` and `app/__init__.py` to load sensitive data (`SECRET_KEY`, `DATABASE_URL`) from environment variables. Create an `.env` file for local development.

2.  **Production Dependencies:** Create the `requirements.txt` and `Procfile` files needed for Gunicorn.

3.  **Appearance Improvement:** Integrate **Bootstrap** into the application, modify `base.html`, and try to improve selected elements (e.g., navigation, footer, or page layout) using Bootstrap.

4.  **(Bonus) Mission Start:** Deploy the application to a service like **PythonAnywhere or Render** or another hosting provider.

---
#### © Jiří Svoboda (George Freedom)
- Web: https://GeorgeFreedom.com
- LinkedIn: https://www.linkedin.com/in/georgefreedom/
- Book me: https://cal.com/georgefreedom